In [1]:
import torch
import torch.nn as nn


In [2]:
input_image=torch.rand(1,3,224,224)

![](MobileNet.png)

https://www.cnblogs.com/darkknightzh/p/9410540.html

In [20]:
def conv_bn(in_ch,out_ch,s):
    return nn.Sequential(nn.Conv2d(in_ch,out_ch,kernel_size=3,stride=s,padding=1,bias=False),
                        nn.BatchNorm2d(out_ch),
                        nn.ReLU(True))

def conv_dw(in_ch,out_ch,s):
    return nn.Sequential(nn.Conv2d(in_ch,in_ch,kernel_size=3,stride=s,padding=1,groups=in_ch,bias=False),
                        nn.BatchNorm2d(in_ch),
                        nn.ReLU(True),
                        nn.Conv2d(in_ch,out_ch,kernel_size=1,stride=1,padding=0,bias=False),
                        nn.BatchNorm2d(out_ch),
                        nn.ReLU(True))

In [23]:
class MobileNet(nn.Module):
    
    def __init__(self,num_class=1000):
        super(MobileNet,self).__init__()
        
        self.feature=nn.Sequential(
            conv_bn(3,32,2),
            conv_dw(32,64,1),
            conv_dw(64,128,2),
            conv_dw(128,128,1),
            conv_dw(128,256,2),
            conv_dw(256,256,1),
            conv_dw(256,512,2),
            conv_dw(512,512,1),
            conv_dw(512,512,1),
            conv_dw(512,512,1),
            conv_dw(512,512,1),
            conv_dw(512,512,1),
            
            conv_dw(512,1024,2),
            conv_dw(1024,1024,1),
            
            nn.AvgPool2d(kernel_size=7,stride=1)
        )
        
        self.classifier=nn.Linear(1024,num_class)
        
    def forward(self,x):
        x=self.feature(x)
        x=x.view(x.size(0),-1)
        x=self.classifier(x)
        
        return x

In [24]:
net=MobileNet(1000)

out=net(input_image)
print(out.shape)

torch.Size([1, 1000])


In [25]:
torch.save(net,'Mobilenet.pth')

F:\anaconda3\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MobileNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
